# An example of applying MSSA to the coefficients you created using pyEXP
This example assumes that you have run the `sample_part1` notebook to create the coefficients.

We begin by importing `pyEXP` and friends and setting the working directory.

In [ ]:
import os
import yaml
import pyEXP
import numpy as np
import matplotlib.pyplot as plt

# In this test, I assume that sample.py has already been run to
# generate a coefficient set.  This script points at that directory
# and does some additional analysis and plotting
#
os.chdir('/home/weinberg/Nbody/Better')

## Create the basis
This step is the same as in the previous notebook.  We are going to use the basis for field evaluation so we need the basis.

In [ ]:
# Get the basis config
#
yaml_config = ""
with open('basis.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
    yaml_config = yaml.dump(config)

# Construct the basis instance
#
basis = pyEXP.basis.Basis.factory(yaml_config)

## Reading the halo coefficients
We created the halo coefficients in the last notebook and stashed them in the EXP HDF5 format.  Now, we'll read them back.

In [ ]:
# Reread the coefs from the file
#
coefs = pyEXP.coefs.Coefs.factory('test_better.h5')

print("Got coefs for name=", coefs.getName())

## Visualizing the fields
We made a few test grids in Part 1 of the notebook.  This time, we'll make surface renderings:

In [ ]:
# Now try some slices for rendering
#
times = coefs.Times()
pmin  = [-1.0, -1.0, 0.0]
pmax  = [ 1.0,  1.0, 0.0]
grid  = [  40,   40,   0]

fields = pyEXP.field.FieldGenerator(times, pmin, pmax, grid)

surfaces = fields.slices(basis, coefs)

print("We now have the following [time field] pairs")
final = 0.0
for v in surfaces:
    print('-'*40)
    for u in surfaces[v]:
        print("{:8.4f}  {}".format(v, u))
        final = v

# Print the potential image at the final time (I think there is a
# fencepost issue in this grid, no matter).
x = np.linspace(pmin[0], pmax[0], grid[0])
y = np.linspace(pmin[1], pmax[1], grid[1])
xv, yv = np.meshgrid(x, y)

# Visualize the final time slice in grid.  Obviously, we could use any 
# field we want.  Here is potential and radial force.

cont1 = plt.contour(xv, yv, surfaces[final]['p'].transpose(), colors='k')
plt.clabel(cont1, fontsize=9, inline=True)
cont2 = plt.contourf(xv, yv, surfaces[final]['p'].transpose())
plt.colorbar(cont2)
plt.xlabel('x')
plt.ylabel('y')
plt.title('Potential at T={}'.format(final))
plt.show()

cont1 = plt.contour(xv, yv, surfaces[final]['fr'].transpose(), colors='k')
plt.clabel(cont1, fontsize=9, inline=True)
cont2 = plt.contourf(xv, yv, surfaces[final]['fr'].transpose())
plt.colorbar(cont2)
plt.xlabel('x')
plt.ylabel('y')
plt.title('Radial force at T={}'.format(final))

## Using MSSA

In [ ]:
# Make a subkey sequence
#
keylst = coefs.makeKeys([1])
print("Keys=", keylst)

config = {"dark halo": (coefs, keylst, [])}

window = int(len(coefs.Times())/2)
npc = 10

print("Window={} PC number={}".format(window, npc))

ssa = pyEXP.mssa.expMSSA(config, window, npc)

ev = ssa.eigenvalues()

plt.plot(ev, 'o-')
plt.xlabel("Index")
plt.ylabel("EV")
plt.title("Eigenvalues by index")
plt.show()

times = coefs.Times();
pc    = ssa.getPC();

rows, cols = pc.shape

for i in range(cols):
    plt.plot(times[0:rows], pc[:,i], '-', label="{:d}".format(i))

plt.xlabel('Time')
plt.ylabel('PC')
plt.legend()
plt.title("Principal components (left-singular vectors)")
plt.show()

# Okay, now try a reconstruction
#
ssa.reconstruct()

newdata = ssa.getReconstructed(False)
print('newdata is a', type(newdata))

# Try the kmeans analysis (not sure this is working correctly yet,
# although it used to work and nothing has changed)
#
ssa.kmeans()

# Test the PNG output
#
ssa.wcorrPNG()